In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("C:\\Users\\argo\\Downloads\\Plastic detection.v1i.multiclass\\train\\_classes.csv", delimiter=",")
dict_plastic = {}

for number, name in enumerate(df.columns):
    if name == 'filename':continue              # skip 1st column
    dict_plastic[number-1] = name.strip()
dict_plastic = {key: dict_plastic[key] for key in dict_plastic if key != 'Unlabeled'}
img_size = 224

In [3]:
def loadImage(path, label, image_size=(224,224), return_label=False):
    labels = pd.read_csv(label)
    labels = labels.sort_values('filename')
    labels = labels.loc[:, labels.columns != 'filename']
    labels = np.array(labels.idxmax(axis=1).str.strip().astype(
            'category').cat.codes).reshape(-1,1)
    
    if return_label:
        return labels
    images = tf.keras.utils.image_dataset_from_directory(path, shuffle=False, batch_size=None, label_mode=None, image_size=image_size)
    labels = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((images, labels))

In [4]:
path_train = "C:\\Users\\argo\\Downloads\\Plastic detection.v1i.multiclass\\train\\"
path_valid = "C:\\Users\\argo\\Downloads\\Plastic detection.v1i.multiclass\\valid\\"
path_test = "C:\\Users\\argo\\Downloads\\Plastic detection.v1i.multiclass\\test\\"
train_dataset = loadImage(path_train, path_train+"_classes.csv", image_size=(img_size,img_size))
valid_dataset = loadImage(path_valid, path_valid+"_classes.csv", image_size=(img_size,img_size))
test_dataset = loadImage(path_test, path_test+"_classes.csv", image_size=(img_size,img_size))
test_labels = loadImage(path_test, path_test+"_classes.csv", return_label=True)

Found 8110 files belonging to 1 classes.
Found 869 files belonging to 1 classes.
Found 435 files belonging to 1 classes.


In [5]:
train = train_dataset.cache('train1').shuffle(500).batch(32)
valid = valid_dataset.cache().batch(32)
test = test_dataset.cache().batch(32)

In [6]:
model2=tf.keras.applications.convnext.ConvNeXtBase(
        model_name='convnext_base',
        include_top=False,
        include_preprocessing=True,
        input_shape=(img_size,img_size,3),
        pooling='max',
    
)
model2.trainable=False

In [7]:
model= tf.keras.models.Sequential([
    model2,
    tf.keras.layers.Dense(len(dict_plastic),"softmax")
])
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer="adam",
    metrics=['accuracy']
)
model.build([None,224,224,3])
model.summary(
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 convnext_base (Functional)  (None, 1024)              87566464  
                                                                 
 dense (Dense)               (None, 23)                23575     
                                                                 
Total params: 87590039 (334.13 MB)
Trainable params: 23575 (92.09 KB)
Non-trainable params: 87566464 (334.04 MB)
_________________________________________________________________


In [8]:
model.fit(
    train,
    validation_data=valid,
    epochs=10

)

Epoch 1/10
  2/254 [..............................] - ETA: 3:25:21 - loss: 2.3911 - accuracy: 0.3281